In [1]:
import pickle

with open('BB', 'rb') as f:
    pub = pickle.load(f)

# 1 #
pub

{'ck': (2360694889179780611464879144859,
  1180347444589890305732439572429,
  1093908988588581740256240460076,
  1147433907296382571186200187057),
 'P': array(['Melissa James', 'Mark Demopoulos', 'Jimmy Gallo', 'Leola Neal'],
       dtype='<U15'),
 'n': 100,
 'Publ': [(1,
   array([[481315325539922063172409050791, 2217294178961342500038839144739,
           1495536867294206894589443493681, 901914969084845335016076218993],
          [826360490767628922687771934435, 1495088458137864467322653419461,
           1193614471311987537650209820935, 1817940380874655774782644996434]],
         dtype=object),
   array([[55409135671797451790817441251, 184991518934170475218699482288,
           195539714325735441078049757489, 1569594927980854895195946677848],
          [35621302824710908411524392824, 1696518882061016847560007122204,
           393824308779865772953763516895, 1258520132279451447000653339521]],
         dtype=object)),
  (2,
   array([[481315325539922063172409050791, 22172941789613425